In [1]:
import geopandas as gpd
import pandas as pd

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


In [2]:
list_kdprov=['16','18']

In [3]:
for kdprov in list_kdprov:
    print('[+] Melakukan proses kalkulasi untuk provinsi {}'.format(kdprov))
    osm=gpd.read_file('data/output/4_RAI/fin_rai_osm_degurba_'+kdprov+'.gpkg')
    rbi=gpd.read_file('data/output/4_RAI/fin_rai_rbi_degurba_'+kdprov+'.gpkg')

    kabkota=gpd.read_file('data/input/Master_Data/kab_20201_byindo.gpkg')
    osm_prov=pd.pivot_table(data=osm,columns='accessed',index='classification',values='pop',aggfunc='sum').fillna(0).assign(
        percent=lambda data_p:data_p[1]*100/(data_p[0]+data_p[1]))
    rbi_prov=pd.pivot_table(data=rbi,columns='accessed',index='classification',values='pop',aggfunc='sum').fillna(0).assign(
        percent=lambda data_p:data_p[1]*100/(data_p[0]+data_p[1]))
    osm_rbi_merge=osm[['Name', 'pop', 'p_builtup', 'is_U_CENTRE_w_BU', 'is_U_CLUSTER',
           'classification', 'accessed', 'geometry']].rename(columns={'accessed':'osm_accessed'}).merge(rbi[['Name','accessed']].rename(
        columns={'accessed':'rbi_accessed'})).to_crs(4326)
    kabkota_osm_rbi=gpd.sjoin(osm_rbi_merge,kabkota[['idkab','geometry']])
    osm_kabkota=pd.pivot_table(data=kabkota_osm_rbi,columns='osm_accessed',index=['idkab','classification'],values='pop',aggfunc='sum').fillna(0).assign(
        percent=lambda data_p:data_p[1]*100/(data_p[0]+data_p[1]))
    rbi_kabkota=pd.pivot_table(data=kabkota_osm_rbi,columns='rbi_accessed',index=['idkab','classification'],values='pop',aggfunc='sum').fillna(0).assign(
        percent=lambda data_p:data_p[1]*100/(data_p[0]+data_p[1]))

    writer = pd.ExcelWriter('data/output/4_RAI/compiled_RAI'+kdprov+'.xlsx', engine='xlsxwriter')
    osm_prov.to_excel(writer, sheet_name='prov_osm')
    rbi_prov.to_excel(writer, sheet_name='prov_rbi')
    osm_kabkota.to_excel(writer, sheet_name='kabkota_osm')
    rbi_kabkota.to_excel(writer, sheet_name='kabkota_rbi')
    writer.save()
    print('[+] Selesai melakukan proses kalkulasi untuk provinsi {}'.format(kdprov))